In [1]:
%pip install --upgrade datasets groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 2.0 MB/s eta 0:00:00MB/s eta 0:00:01
  Attempting uninstall: groq
    Found existing installation: groq 0.5.0
    Uninstalling groq-0.5.0:
      Successfully uninstalled groq-0.5.0
Note: you may need to restart the kernel to use updated packages.


In [16]:
from datasets import Dataset, load_dataset

In [17]:
INPUT_DATASET = "dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles"
SYSTEM_PROMPT = """
Como un experto en derecho y leyes españolas, tu tarea es crear preguntas y respuestas sobre el Boletín Oficial del Estado (BOE) de España.

Para ello, debes tener en cuenta y utilizar el contexto proporcionado para generar una pregunta y su correspondiente respuesta por cada párrafo.

Asegúrate de escribir siempre en español.

El formato de la salida JSON será:

[
  {"question": "text for question", "answer": "text for the answer"},
  {"question": "text for question", "answer": "text for the answer"}
]
"""

In [18]:
dataset = load_dataset(INPUT_DATASET)

Generating train split:   0%|          | 0/169 [00:00<?, ? examples/s]

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'text'],
        num_rows: 169
    })
})

In [20]:
dataset['train'][0]['text']

'Artículo 1\n1. España se constituye en un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico la libertad, la justicia, la igualdad y el pluralismo político.\n\n2. La soberanía nacional reside en el pueblo español, del que emanan los poderes del Estado.\n\n3. La forma política del Estado español es la Monarquía parlamentaria.'

In [21]:
from groq import Groq
from google.colab import userdata
import json
import time


client = Groq(
    api_key=os.environ.get('GROQ_API_KEY'),
)

In [22]:
data = []
for article in dataset['train']:
  time.sleep(2.5)  # groq limits: https://console.groq.com/settings/limits

  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": article['text'],
        },
      ],
      model="llama3-70b-8192",
      temperature=0,
      stream=False,
  )

  try:
    responses = json.loads(chat_completion.choices[0].message.content)
    for response in responses:
      row = {}
      row['number'] = article['article']
      row['context'] = article['text']
      row['question'] = response['question']
      row['answer'] = response['answer']
      data.append(row)
  except:
    print("Wrong generated")
    print(f"Article {article['article']}")
    print(chat_completion.choices[0].message.content)

Wrong generated
Article 5
[
  {"question": "¿Cuál es la capital del Estado según el Boletín Oficial del Estado (BOE) de España?", "answer": "La villa de Madrid."},
]
Wrong generated
Article 14
[
  {"question": "¿Cuál es el principio fundamental que establece la igualdad de los españoles ante la ley?", "answer": "La igualdad ante la ley sin discriminación alguna por razón de nacimiento, raza, sexo, religión, opinión o cualquier otra condición o circunstancia personal o social."},
]
Wrong generated
Article 26
[
  {"question": "¿Qué tipo de tribunales están prohibidos en el ámbito de la Administración civil y de las organizaciones profesionales?", "answer": "Los Tribunales de Honor"},
]
Wrong generated
Article 42
[
  {"question": "¿Cuál es el deber del Estado según el artículo 42 en relación con los trabajadores españoles en el extranjero?", "answer": "Velar especialmente por la salvaguardia de sus derechos económicos y sociales y orientar su política hacia su retorno."},
]
Wrong generate

In [23]:
data[0]

{'number': 1,
 'context': 'Artículo 1\n1. España se constituye en un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico la libertad, la justicia, la igualdad y el pluralismo político.\n\n2. La soberanía nacional reside en el pueblo español, del que emanan los poderes del Estado.\n\n3. La forma política del Estado español es la Monarquía parlamentaria.',
 'question': '¿Cuáles son los valores superiores del ordenamiento jurídico en España?',
 'answer': 'La libertad, la justicia, la igualdad y el pluralismo político.'}

In [24]:
# Convertir la lista de diccionarios a un diccionario de listas
data_dict = {key: [dic[key] for dic in data] for key in data[0]}

In [25]:
dataset = Dataset.from_dict(data_dict)

In [26]:
dataset

Dataset({
    features: ['number', 'context', 'question', 'answer'],
    num_rows: 515
})

In [27]:
import huggingface_hub

huggingface_hub.login()

In [28]:
dataset.push_to_hub(f"{INPUT_DATASET}-qa")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa/commit/46e19eba0def56789cfb7ed3ad879e1f67aabe7e', commit_message='Upload dataset', commit_description='', oid='46e19eba0def56789cfb7ed3ad879e1f67aabe7e', pr_url=None, pr_revision=None, pr_num=None)